In [2]:
# los más de los duros, los reales, los meros buenos
import pandas as pd
import numpy as np

# para graficar
import matplotlib.pyplot as plt
import seaborn as sns   

# libreria que usaremos para aprendizaje
import sklearn

# para preprocesamiento
from sklearn.feature_extraction import DictVectorizer

# los modelos
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

# las metricas
from sklearn.metrics import mean_squared_error

In [3]:
yellow_taxi_jan = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [4]:
yellow_taxi_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [5]:
yellow_taxi_jan.shape

(3066766, 19)

R: hay 19 columnas

Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [6]:
yellow_taxi_jan.tpep_dropoff_datetime = pd.to_datetime(yellow_taxi_jan.tpep_dropoff_datetime) 
yellow_taxi_jan.tpep_pickup_datetime = pd.to_datetime(yellow_taxi_jan.tpep_pickup_datetime)

yellow_taxi_jan["duration"] = yellow_taxi_jan.tpep_dropoff_datetime - yellow_taxi_jan.tpep_pickup_datetime
yellow_taxi_jan["duration"] = yellow_taxi_jan.duration.apply(lambda x: x.total_seconds()/60)

In [7]:
yellow_taxi_jan["duration"].std()

42.594351241920904

R: 42.59 min

Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [8]:
rows_before = yellow_taxi_jan.shape[0]
yellow_taxi_jan = yellow_taxi_jan[(yellow_taxi_jan.duration >= 1) & (yellow_taxi_jan.duration <=60)]
rows_after = yellow_taxi_jan.shape[0]
fraction = rows_after/rows_before
fraction

0.9812202822125979

Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [9]:
categorical = ['PULocationID', 'DOLocationID']
yellow_taxi_jan[categorical]  = yellow_taxi_jan[categorical].astype(str)
train_dicts = yellow_taxi_jan[categorical].to_dict(orient="records")

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [10]:
target = 'duration'
y_train = yellow_taxi_jan[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train_lr = lr.predict(X_train)

mean_squared_error(y_train, y_pred_train_lr, squared=False)

7.649261027792376

Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [ ]:
df_val = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
df_val[categorical] = df_val[categorical].astype(str)

val_dicts = df_val.to_dict(orient="records")
X_val = dv.transform(val_dicts)

y_pred_val_lr = lr.predict(X_val)

y_val = df_val[target].values

mean_squared_error(y_val, y_pred_val_lr)